In [1]:
from matplotlib import colors    ### 까만 노드 인력척력 djqt는 버전 + grid_to_graph_woblack 함수 추가 + 1.414 주석처리 + np array 로 변경
import matplotlib.pyplot as plt
import numpy as np
import math
from sklearn.cluster import DBSCAN # conda install -c conda scikit-learn
import pandas as pd # conda install pandas

cmap = colors.ListedColormap(
        [
            '#000000', # 0 검은색
            '#0074D9', # 1 파란색
            '#FF4136', # 2 빨간색
            '#2ECC40', # 3 초록색
            '#FFDC00', # 4 노란색
            '#AAAAAA', # 5 회색
            '#F012BE', # 6 핑크색
            '#FF851B', # 7 주황색
            '#7FDBFF', # 8 하늘색
            '#870C25', # 9 적갈색
            '#505050', # 10 검은색_select
            '#30A4F9', # 11 파란색_select
            #'#FF4136', 
            '#FF7166', # 12 빨간색_select
            '#5EFC70', # 13 초록색_select
            '#FFFC30', # 14 노란색_select
            '#DADADA', # 15 회색_select
            '#F042EE', # 16 핑크색_select
            '#FFB54B', # 17 주황색_select
            '#AFFBFF', # 18 하늘색_select
            '#B73C55'  # 19 적갈색_select
        ])
    #norm = colors.Normalize(vmin=0, vmax=9)
norm = colors.Normalize(vmin=0, vmax=19)

class Pnode :
    def __init__(self, grid, i, j):
        self.color = grid[i][j]
        self.number = self.node_number(len(grid[0]), i, j)
        self.visual_coord = [3 * j, 3 * (len(grid) - i - 1)]  ## coordinate for visualize
        self.coordinate = [j,i]         ## coordinate from the grid
        self.input = 0
        self.output = 0
        self.type = "Pnode"
    def node_number (self, col, i, j): ## start from 0 to (col * row -1) # may not needed
        temp = i * (col) + j
        return temp
    def __str__(self):
        if self.input == 1:
            return f"Pnode: N:{self.number}, I"
        else :
            return f"Pnode: N:{self.number}, O"
class Onode:
    def __init__(self, obj, condition):
        Pnode_list = []
        color_set = set()
        number_set = set()
        for Pnode in obj:
            Pnode_list.append(Pnode)
            color_set.add(Pnode.color)
            number_set.add(Pnode.number)
        self.Pnode_list = Pnode_list
        self.color = color_set            ##questionalbe 
        self.number = number_set  
        self.coordinate = [0,0]
        self.input = 0
        self.output = 0
        self.type = "Onode"
        self.condition = condition
        # [get_coordinate(node) for node in get_center_nodes]    ##questionalbe -> need bbox function first and type will be {(int, int), (int, int) ...}
    def __str__(self):
        pnodes = []
#        for pnode in self.Pnode_list:
#            pnodes.append(pnode.__str__())
        if self.input == 1:
            return f"Onode: N:{self.number}, I"
        else :
            return f"Onode: N:{self.number}, O"
    
class Gnode:
    def __init__(self, node_list): # node_list should contain all the Pnode and Onode from the grid
        self.Node_list = node_list
        color_s = set()
        Pnode_list = []
        Onode_list = []
        for n in node_list:
            if isinstance(n, Pnode):
                color_s.add(n.color)
                Pnode_list.append(n)
            elif isinstance(n, Onode):
                Onode_list.append(n)
        self.color = color_s
        self.Pnode_list = Pnode_list
        self.Onode_list = Onode_list
        self.coordinate = [0,0]
        self.number = 0
        self.input = 0
        self.output = 0
        self.type = "Gnode"
        self.condition = "Gnode"
        # [get_coordinate(node) for node in get_center_nodes]       ## questionable # do we need coornidate for Gnode?
    def __str__(self):
        if self.input == 1:
            return f"Gnode: N:{self.number}, I"
        else :
            return f"Gnode: N:{self.number}, O"

class Vnode:
    def __init__(self, Gnode1, Gnode2): # node_list should contain all the Pnode and Onode from the grid
        self.Gnode_list = [Gnode1, Gnode2]
        self.type = "Vnode"
        self.input = 0
        self.output = 0
        self.color = Gnode1.color.union(Gnode2.color)
        self.Onode_list = [node for node in Gnode1.Onode_list] + [node for node in Gnode2.Onode_list]
        self.Pnode_list = [node for node in Gnode1.Pnode_list] + [node for node in Gnode2.Pnode_list]
    def __str__(self):
        return f"Vnode"

    
class Edge:
    def __init__(self, tag, node1, node2 = None):
        if node2 == None :
            self.node_set = [node1]
        else :
            self.node_set = [node1, node2]
        self.tag = tag       
    def __str__(self):
        n_set = []
        for n in self.node_set:
            n_set.append(n.__str__())
        return f"{n_set, self.tag}"    

def Grid_to_Img(grid): ## function for visualize the image, this function is not a DSL
    plt.axis("off")
    plt.imshow(grid, cmap = cmap, norm = norm)

def Make_NodeList (grid):     ## this function now generate Pnode list from grid
    node_list = []
    if type(grid[0]) == list :
        for i in range(len(grid)):
            for j in range(len(grid[0])):
                temp_node = Pnode(grid, i, j)
                temp_node.color = grid[i][j]
                node_list.append(temp_node)
    return node_list

def Concat_node_list (node_list1, node_list2):
    return node_list1 + node_list2

def Make_Onode (node_list, color_same, dist_dsl, edge_list):
    condition = []
    def edge_list_to_graph(edges):
        graph = {}
        for edge in edges:
            node1, node2 = edge.node_set
            if node1 not in graph:
                graph[node1] = []
            if node2 not in graph:
                graph[node2] = []
            graph[node1].append(node2)
            graph[node2].append(node1)
        return graph
    
    if color_same == True:
        same_color = lambda x1, x2 : True if x1.color == x2.color else False
        node_list1 = node_list
        condition.append("single_color")
    else :
        same_color = lambda x1, x2 : True
        condition.append("multi_color")
        node_list1 = get_background_color_removed(node_list)
    if dist_dsl != None:
        dist_1 = lambda x1, x2 : True if dist_dsl(x1, x2) == 1 else False
        condition.append(dist_dsl.__name__)
    else :
        dist_1 = lambda x1, x2 : True
        condition.append("no_distance")
    same_color_and_dist1 = lambda x1, x2: True if (same_color(x1,x2) == True and dist_1(x1,x2) == True) else False
    s_d_edge_list = Make_edge_list(node_list1, same_color_and_dist1)
    graph = edge_list_to_graph(s_d_edge_list)

    condition = tuple(condition)
    tag = ("get_onode", condition)

    for node in node_list1:
        # if isinstance(node, Onode):
        #     continue
        if node not in graph.keys():
            graph[node] = []
    
    def cluster_graph(graph):
        clusters = []
        visited = set()

        def dfs(node, cluster):
            visited.add(node)
            cluster.add(node)
            for neighbor in graph[node]:
                if neighbor not in visited:
                    dfs(neighbor, cluster)

        for node in graph:
            if node not in visited:
                new_cluster = set()
                dfs(node, new_cluster)
                clusters.append(new_cluster)
                
        return clusters

    clusters = cluster_graph(graph)

    Onode_list = []
    # tag = ("get_onode", None)
    for obj in clusters:
        onode = Onode(obj, condition)
        Onode_list.append(onode)
        for pnode in obj:
            e = Edge(tag, pnode, onode)
            edge_list.append(e)



    return Onode_list, edge_list 

def Make_Gnode (node_list, edge_list):
    gnode = Gnode(node_list)
    gnode_list = []
    tag = ("get_gnode", None)
    for node in node_list:
        e = Edge(tag, gnode, node)
        edge_list.append(e)
    gnode_list.append(gnode)
    return node_list + gnode_list, edge_list

def Make_Vnode (node_list, edge_list):
    gnode_list = []
    for n in node_list:
        if isinstance(n, Gnode):
            gnode_list.append(n)
    assert len(gnode_list) == 2
    vnode = [Vnode(gnode_list[0], gnode_list[1])]
    tag = ("get_vnode", None)
    e1 = Edge(tag, vnode[0], gnode_list[0])
    e2 = Edge(tag, vnode[0], gnode_list[1])
    edge_list.append(e1)
    edge_list.append(e2)
    return node_list + vnode, edge_list

def create_edge_list ():
    return []


    
def create_edge (get_dsl, node1, node2):   ## no self connecting edge
    is_dsl = get_to_is(get_dsl) 
    result = is_dsl(node1, node2)   ## result = (bool (dsl_name, taget))
    if result[0] == False :
        return None
    else :
        tag = result[1]
        edge = Edge(tag, node1, node2)
        return edge



def Make_edge_list (node_list, dsl) :
    edge_list = create_edge_list()
    try :
        if isinstance (dsl(node_list[0]), bool) == True: ## dsl is is_dsl with only one param
            # print("dsl is returning bool type with 1 param")
            tag = (dsl.__name__, None)
            for n1 in node_list:
                if dsl(n1) == True :
                    e = Edge(tag, n1, n1)
                    edge_list.append(e)
            return edge_list
        else :
            pass
    except :
        pass
    try :
        if isinstance (dsl(node_list[0], node_list[0]), bool) == True : ## dsl is is_dsl with two param
            # print("dsl is returning bool type with 2 param")
            tag = (dsl.__name__, None)
            for n1 in node_list:
                for n2 in node_list:
                    if dsl(n1, n2) == True and n1 != n2 :
                        e = Edge(tag, n1, n2)
                        edge_list.append(e)
            return edge_list
    except: 
        pass
    ## dsl is get_dsl
    # print("dsl is get_dsl")
    for n1 in node_list:
        for n2 in node_list:
            e = create_edge(dsl, n1, n2)
            if e != None and n1 != n2:
                edge_list.append(e)
                # print(e)
    return edge_list

def Concat_edge_list (edge_list1, edge_list2):
    return edge_list1 + edge_list2

def get_to_is (get_f):
    param_num = get_f.__code__.co_argcount
    dsl_name = get_f.__name__
    if param_num == 2:
        is_dsl = lambda x1, x2 : (
            (True, (dsl_name, get_f(x1, x2))) 
            if x1 != x2 and get_f(x1, x2) != None
            else (False, None)
            )## (bool (name, taget))
        
    elif param_num == 1:
        is_dsl = lambda x1, x2: (
            (True, (dsl_name, get_f(x1))) 
            if (
                get_f(x1) == get_f(x2) and get_f(x1) != None
                and x1 != x2 
            )
            else (
                (True, (dsl_name, None)) if (isinstance(get_f(x1), list) and x2 in get_f(x1))
                else (False, None)
            )
        )
    else :
        print("exception has occured")
        return
    return is_dsl

def node_list_numbering (node_list, i):
    if i == 0: ## input
        for n in node_list:
            if isinstance(n, Gnode):
                n.number = 1
            n.input = 1
            n.output = 0
    else : # output
        for n in node_list:
            if isinstance(n, Gnode):
                n.number = 2
            n.input = 0
            n.output = 1

def visualize(node_list, edge_list, tag):
    x = []
    y = []
    colors = []
    num_nodes = len(node_list)
    for ele in (node_list):
        x.append(ele.visual_coord[0])
        y.append(ele.visual_coord[1])
        colors.append(ele.color)
    for i in range(len(node_list)):
        plt.text(x[i] - 0.1, y[i]- 0.1, node_list[i].number , size = 15, color = 'white')

    for i in range(num_nodes) :
        for j in range(i, num_nodes) :
            for edge in edge_list:
                if edge.node_set == {node_list[i], node_list[j]} and edge.tag == tag:
                    plt.plot([x[i], x[j]], [y[i], y[j]], color = 'black', linewidth = 3)
    plt.axis('off')
    plt.axis('equal')
    plt.scatter(x, y, s = 500, c = colors, cmap = cmap, norm = norm)
    plt.show()

import math

#P-layer DSLs 
def get_color_of_node(node):   
    if isinstance(node, Vnode):
        return None
    # if node.color == 0:
    #     return None
    return node.color
def get_horizontal_index(node):  
    if isinstance(node, Pnode):
        return node.coordinate[1]
    else :
        return None
def get_vertical_index(node):    
    if isinstance(node, Pnode):
        return node.coordinate[0]
    else: 
        return None
def get_polar_distance(node1, node2):  
    if isinstance(node1, Pnode) and isinstance(node2, Pnode):
        x = abs(node1.coordinate[0] - node2.coordinate[0])
        y = abs(node1.coordinate[1] - node2.coordinate[1])
        return x if x > y else y
    else :
        return None
def get_manhattan_dist (node1,node2) :     
    if isinstance(node1, Pnode) and isinstance(node2, Pnode):
        x = abs(node1.coordinate[0] - node2.coordinate[0])
        y = abs(node1.coordinate[1] - node2.coordinate[1])
        dist = x + y
        return dist
    else : 
        return None

def get_coordinate(node):     
    if isinstance(node, Pnode):
        return node.coordinate
    else :
        return None
def get_dimension_diff(Xnode1, Xnode2): 
    dimensions1 = get_dimension(Xnode1)
    dimensions2 = get_dimension(Xnode2)
    if dimensions1 == (1,1) or dimensions1 == None:
        return None                         
    if dimensions2 == (1,1) or dimensions2 == None:
        return None     
    try :
        width_diff = dimensions2[1] - dimensions1[0]
        height_diff = dimensions1[1] - dimensions2[0]
        return (width_diff, height_diff)
    except :
        return None

def get_color_difference_set(Xnode1, Xnode2): 
    if isinstance(Xnode1, Vnode) or isinstance(Xnode2, Vnode):
        return None
    colors1 = Xnode1.color
    colors2 = Xnode2.color
    if isinstance(colors1, int):
        return None                          
    if isinstance(colors2, int):
        return None     
    color_diff = [(colors1 - colors2), (colors2 - colors1)]
    return color_diff

def get_component(onode):                   
    return onode.Pnode_list

##def get_bounding_box(Pnode_list, Onode)


##G-node layer DSLs
def get_dominant_color(i_value):  
    color_counts = {}
    if isinstance(i_value, Gnode) or isinstance(i_value, Onode):   
        node_list = i_value.Pnode_list
    elif isinstance(i_value, Pnode) or isinstance(i_value, Vnode):
        return None
    else :
        node_list = i_value   
    for node in node_list: color_counts[node.color] = color_counts.get(node.color, 0) + 1
    return max(color_counts, key=color_counts.get)  

def get_background_color_removed(i_value):
    if isinstance(i_value, Gnode):
        pnode_list = i_value.Pnode_list
        node_list = Gnode.node_list
    else :
        node_list = i_value
        pnode_list = []
        for n in i_value:
            if isinstance(n, Pnode):
                pnode_list.append(n)
    dominant_color = get_dominant_color(pnode_list)
    return [node for node in node_list if node.color != dominant_color]

    
    return filtered_nodes

def get_least_common_color(Xnode):               
    color_counts = {}
    if isinstance(Xnode, Pnode) or isinstance(Xnode, Vnode):
        return None
    node_list = Xnode.Pnode_list
    for node in node_list:
        color = get_color_of_node(node)  
        color_counts[color] = color_counts.get(color, 0) + 1
    min_count = min(color_counts.values())
    return [color for color, count in color_counts.items() if count == min_count]
def get_width(Xnode):
    if isinstance(Xnode, Pnode) or isinstance(Xnode, Vnode):
        return None
    "DSL for getting the width of a node-list"
    # print(Xnode)
    # print(Xnode.Pnode_list)
    node_list = Xnode.Pnode_list
    max_v = max(node.coordinate[0] for node in node_list)
    min_v = min(node.coordinate[0] for node in node_list)
    return max_v - min_v + 1
def get_height(Xnode):              
    "DSL for getting the height of a node-list"
    if isinstance(Xnode, Pnode) or isinstance(Xnode, Vnode):
        return None
    node_list = Xnode.Pnode_list
    max_v = max(node.coordinate[1] for node in node_list)
    min_v = min(node.coordinate[1] for node in node_list)
    return max_v - min_v + 1
def get_dimension(Xnode):           
    if isinstance(Xnode, Pnode):
        return (1,1)
    if isinstance(Xnode, Vnode):
        return None
    node_list = Xnode.Pnode_list
    width = get_width(Xnode)
    height = get_height(Xnode)
    return (width, height)
def get_number_of_nodes(Xnode):      
    if isinstance(Xnode, Pnode):
        return 1
    elif isinstance(Xnode, Vnode):
        return None  #######################################
    node_list = Xnode.Pnode_list
    return len(node_list)

def get_corner(Xnode):
    if isinstance(Xnode, Pnode) or isinstance(Xnode, Vnode):
        return None
    
    node_list = Xnode.Pnode_list
    min_x = min(node.coordinate[0] for node in node_list)
    max_x = max(node.coordinate[0] for node in node_list)
    min_y = min(node.coordinate[1] for node in node_list)
    max_y = max(node.coordinate[1] for node in node_list)

    # Determine the corner nodes based on the bounding box
    corners = {(min_x, min_y), (min_x, max_y), (max_x, min_y), (max_x, max_y)}
    return [node for node in node_list if tuple(node.coordinate) in corners]



######################################################################
def get_non_margin(Xnode):                
    if isinstance(Xnode, Pnode) or isinstance(Xnode, Vnode):
        return None
    margin_nodes = []
    node_list = Xnode.Pnode_list
    width = get_width(Xnode)
    height = get_height(Xnode)
    for node in node_list:
        x, y = get_coordinate(node)
        if x == 0 or y == 0 or x == width - 1 or y == height - 1:
            margin_nodes.append(node)
    return margin_nodes
# def get_margin(Xnode):                
#     if isinstance(Xnode, Pnode) or isinstance(Xnode, Vnode):
#         return None
#     node_list = Xnode.Pnode_list
#     return [node for node in node_list if node not in get_non_margin(Xnode)]
############################################################################
## reject! what if Xnode is a instance of Onode.



def get_center_nodess(Xnode):      
    if isinstance(Xnode, Pnode):
        # For a single Pnode, it is its own center node
        return [Xnode]
    elif isinstance(Xnode, Onode):
        # For an Onode, return the center Pnodes in its Pnode list
        center_nodes = []
        width = get_width(Xnode)
        height = get_height(Xnode)
        center_x = width // 2 if width % 2 == 0 else width // 2 + 1
        center_y = height // 2 if height % 2 == 0 else height // 2 + 1
        for node in Xnode.Pnode_list:
            if node.coordinate == [center_x, center_y]:
                center_nodes.append(node)
        if len(center_nodes) != 0:
            return center_nodes
        else :
            return None
    elif isinstance(Xnode, Gnode):
        # For a Gnode, return the center Pnodes from its Pnode list
        center_nodes = []
        width = get_width(Xnode)
        height = get_height(Xnode)
        center_x = width // 2 if width % 2 == 0 else width // 2 + 1
        center_y = height // 2 if height % 2 == 0 else height // 2 + 1
        for node in Xnode.Pnode_list:
            if node.coordinate == [center_x, center_y]:
                center_nodes.append(node)
        return center_nodes
    else:
        return None
def get_specific(Gnode, target_colors): 
    node_list = Gnode.Pnode_list
    return [node for node in node_list if node.color in target_colors]

def get_height_difference(Xnode1, Xnode2):  
    height1 = get_height(Xnode1) 
    height2 = get_height(Xnode2)
    if height1 == None or height2 == None:
        return None
    return abs(height1 - height2)

def get_width_difference(Xnode1, Xnode2):  
    width1 = get_width(Xnode1)
    width2 = get_width(Xnode2)
    if width1 == None or width2 == None:
        return None
    return abs(width1 - width2)

def get_max_height(Xnode1, Xnode2):      
    return max(get_height(Xnode1), get_height(Xnode2))    

def get_max_width(Xnode1, Xnode2):
    return max(get_width(Xnode1), get_width(Xnode2))   


###########################################################################
def get_bounding_boxx(onode):
    if not isinstance(onode, Onode):
        return None
    min_x = min_y = float('inf')
    max_x = max_y = float('-inf')
    for pnode in onode.Pnode_list:
        x, y = pnode.coordinate
        min_x = min(min_x, x)
        min_y = min(min_y, y)
        max_x = max(max_x, x)
        max_y = max(max_y, y)
    return min_x, min_y, max_x, max_y

def get_margin(onode):
    if not isinstance(onode, Onode):
        return None
    margin_nodes = []
    min_x, min_y, max_x, max_y = get_bounding_boxx(onode)
    for pnode in onode.Pnode_list:
        x, y = pnode.coordinate
        if x == min_x or x == max_x or y == min_y or y == max_y:
            margin_nodes.append(pnode)
    return margin_nodes
    
def equal_margin(onode):
    if not isinstance(onode, Onode):
        return None
    equal_margin_onodes = []
    for onode in onode:
        margin_nodes = get_margin(onode)
        if len(margin_nodes) == len(onode.Pnode_list) and \
                all(pnode.color == onode.Pnode_list[0].color for pnode in onode.Pnode_list) and \
                len(onode.Pnode_list) > 7:
            equal_margin_onodes.append(onode)
    return equal_margin_onodes

def is_ring(onode):
    if not isinstance(onode, Onode):
        return False
    margin_nodes = get_margin(onode)
    return len(margin_nodes) == len(onode.Pnode_list) and \
           all(pnode.color == onode.Pnode_list[0].color for pnode in onode.Pnode_list) and \
           len(onode.Pnode_list) > 7

def is_rectangle(onode):
    if not isinstance(onode, Onode):
        return False
    margin_nodes = get_margin(onode)
    if len(margin_nodes) != 4:
        return False
    x_values = [pnode.coordinate[0] for pnode in margin_nodes]
    y_values = [pnode.coordinate[1] for pnode in margin_nodes]
    # Check if the x coordinates form a rectangle
    if len(set(x_values)) != 2:
        return False
    # Check if the y coordinates form a rectangle
    if len(set(y_values)) != 2:
        return False
    return True

def is_square(onode):
    if not isinstance(onode, Onode):
        return False
    # Get the bounding box coordinates
    min_x, min_y, max_x, max_y = get_bounding_boxx(onode)
    # Calculate the width and height of the bounding box
    width = max_x - min_x + 1
    height = max_y - min_y + 1
    # Check if all sides are equal in length and if it's at least 2 by 2
    return width == height and width >= 2

def get_cross_properties(onode):
    if not isinstance(onode, Onode):
        return None
    min_x, min_y, max_x, max_y = get_bounding_boxx(onode)
    width = max_x - min_x + 1
    height = max_y - min_y + 1
    mid_x = (min_x + max_x) // 2
    mid_y = (min_y + max_y) // 2
    return width, height, mid_x, mid_y

def is_cross(onode):
    if not isinstance(onode, Onode):
        return False
    width, height, mid_x, mid_y = get_cross_properties(onode)
    if width <= 2 or height <= 2:
        return False
    has_horizontal_line = False
    has_vertical_line = False
    for pnode in onode.Pnode_list:
        x, y = pnode.coordinate
        if y == mid_y:
            has_horizontal_line = True
        if x == mid_x:
            has_vertical_line = True
    return has_horizontal_line and has_vertical_line

def is_symmetric(onode):
    if not isinstance(onode, Onode):
        return False
    width, height, mid_x, mid_y = get_cross_properties(onode)
    # Check if the Onode is symmetric along both the horizontal and vertical axes
    for pnode in onode.Pnode_list:
        x, y = get_coordinate(pnode)
        if (x != mid_x and y != mid_y) or (x == mid_x and y == mid_y):
            return False  # If any node doesn't have its symmetric counterpart, return False
    
    return True  

def is_contained(onode):##not working will fix it  ????????????????
    nodes = onode.Pnode_list
    filtered_onodes = [node for node in node_list if isinstance(node, Onode)]
    for node in nodes:
        for other_onode in node_list:
            if other_onode is not onode and isinstance(other_onode, Onode):
                other_nodes = other_onode.Pnode_list
                if all(other_node in nodes for other_node in other_nodes):
                    return True
    return False

#############################################################################




###P-O Layer DSL

In [2]:
dsl_player_only = [get_horizontal_index, get_vertical_index, get_polar_distance, get_manhattan_dist]
dsl_general = [get_coordinate, get_dimension_diff, get_color_difference_set, get_dominant_color,
               get_least_common_color, get_width, get_height, get_dimension, get_number_of_nodes,
               get_corner, get_center_nodess, get_height_difference, get_width_difference]
is_dsl = [is_ring, is_rectangle, is_square, is_symmetric]
is_dsl_question = [is_cross, is_contained]
other_funtion = [get_background_color_removed, get_specific, get_max_height, get_max_width]
isuue_function = [get_margin, get_non_margin, get_color_of_node]

def Make_test_KG(task):
    test_grid = task['test'][0]["input"]
    node_list = Make_NodeList(test_grid)
    edge_list = create_edge_list()

    for dsl in dsl_player_only:
        temp_edge_list = Make_edge_list(node_list, dsl)
        edge_list = Concat_edge_list(temp_edge_list, edge_list)

        temp_edge_list = Make_edge_list(node_list, dsl)
        edge_list = Concat_edge_list(temp_edge_list, edge_list)
        
    temp_node_list, edge_list = Make_Onode(node_list, True, get_polar_distance, edge_list)
    concat_list = Concat_node_list(node_list, temp_node_list)
    temp_node_list, edge_list = Make_Onode(node_list, False, get_polar_distance, edge_list)
    concat_list = Concat_node_list(concat_list, temp_node_list)

    temp_node_list, edge_list = Make_Onode(node_list, True, get_manhattan_dist, edge_list)
    concat_list = Concat_node_list(concat_list, temp_node_list)
    temp_node_list, edge_list = Make_Onode(node_list, False, get_manhattan_dist, edge_list)
    concat_list = Concat_node_list(concat_list, temp_node_list)

    temp_node_list, edge_list = Make_Onode(node_list, True, None, edge_list)
    concat_list = Concat_node_list(concat_list, temp_node_list)
    temp_node_list, edge_list = Make_Onode(node_list, False, None, edge_list)
    concat_list = Concat_node_list(concat_list, temp_node_list)

    node_list = concat_list
    # for n in input_node_list:
    #     print(n)


    node_list, edge_list = Make_Gnode(node_list, edge_list)
    node_list_numbering(node_list, 0)

    wo_background_nodelist = get_background_color_removed(node_list) 

    ## is_same_color should not draw edge between background color Pnodes
    temp_edge_list = Make_edge_list(wo_background_nodelist, get_color_of_node)

    edge_list = Concat_edge_list(temp_edge_list, edge_list)

    for dsl in dsl_general + is_dsl:
        temp_edge_list = Make_edge_list(node_list, dsl)
        edge_list = Concat_edge_list(temp_edge_list, edge_list)
        
    return [node_list, edge_list]

def Make_KG(task):
    num_pairs = len(task["train"])
    input_grids = []
    output_grids = []
    for grid in task["train"]:
        input_grids.append(grid["input"])
    for grid in task["train"]:
        output_grids.append(grid["output"])
        

    KGs = []
    for i in range(num_pairs):
        print("converting task to knowledge graph...", i + 1, "/", num_pairs)
        input_node_list = Make_NodeList(input_grids[i])
        input_edge_list = create_edge_list()
        
        output_node_list = Make_NodeList(output_grids[i])
        output_edge_list = create_edge_list()

        for dsl in dsl_player_only:
            temp_edge_list = Make_edge_list(input_node_list, dsl)
            input_edge_list = Concat_edge_list(temp_edge_list, input_edge_list)

            temp_edge_list = Make_edge_list(output_node_list, dsl)
            output_edge_list = Concat_edge_list(temp_edge_list, output_edge_list)

        ## modify here to add more various type of Onode
        temp_node_list, input_edge_list = Make_Onode(input_node_list, True, get_polar_distance, input_edge_list)
        concat_list = Concat_node_list(input_node_list, temp_node_list)
        temp_node_list, input_edge_list = Make_Onode(input_node_list, False, get_polar_distance, input_edge_list)
        concat_list = Concat_node_list(concat_list, temp_node_list)

        temp_node_list, input_edge_list = Make_Onode(input_node_list, True, get_manhattan_dist, input_edge_list)
        concat_list = Concat_node_list(concat_list, temp_node_list)
        temp_node_list, input_edge_list = Make_Onode(input_node_list, False, get_manhattan_dist, input_edge_list)
        concat_list = Concat_node_list(concat_list, temp_node_list)

        temp_node_list, input_edge_list = Make_Onode(input_node_list, True, None, input_edge_list)
        concat_list = Concat_node_list(concat_list, temp_node_list)
        temp_node_list, input_edge_list = Make_Onode(input_node_list, False, None, input_edge_list)
        concat_list = Concat_node_list(concat_list, temp_node_list)

        input_node_list = concat_list
        # for n in input_node_list:
        #     print(n)


        input_node_list, input_edge_list = Make_Gnode(input_node_list, input_edge_list)

        ## modify here to add more various type of Onode
        temp_node_list, output_edge_list = Make_Onode(output_node_list, True, get_polar_distance, output_edge_list)
        output_node_list = Concat_node_list(output_node_list, temp_node_list)
        output_node_list, output_edge_list = Make_Gnode(output_node_list, output_edge_list)

        node_list_numbering(input_node_list, 0)
        node_list_numbering(output_node_list, 1)

        pair_node_list = Concat_node_list(input_node_list, output_node_list)
        pair_edge_list = Concat_edge_list(input_edge_list, output_edge_list)
        pair_node_list, pair_edge_list = Make_Vnode(pair_node_list, pair_edge_list)

        wo_background_nodelist = get_background_color_removed(pair_node_list) 

        ## is_same_color should not draw edge between background color Pnodes
        temp_edge_list = Make_edge_list(wo_background_nodelist, get_color_of_node)

        pair_edge_list = Concat_edge_list(temp_edge_list, pair_edge_list)
        #print(get_color_of_node.__name__)

        num_dsl = len(dsl_general + is_dsl)
        for i, dsl in enumerate(dsl_general + is_dsl) :
            # print(dsl.__name__)
            progress = (i + 1) / num_dsl * 100
            bar_width = int(progress)
            print('\r [%-*s] %.1f%%' % (100, '=' * bar_width, progress), end='')

            temp_edge_list = Make_edge_list(pair_node_list, dsl)
            pair_edge_list = Concat_edge_list(temp_edge_list, pair_edge_list)




        KGs.append([pair_node_list, pair_edge_list])
        print()
    print('\r ')
    return KGs

In [3]:
def print_adj(kg):
    node_list1 = kg[0]
    edge_list1 = kg[1]
    result1 = [[[0] for i in range(len(node_list1))] for j in range(len(node_list1))]
    for i, edge in enumerate(edge_list1):
        n1, n2 = edge.node_set
        i1 = node_list1.index(n1)
        i2 = node_list1.index(n2)
        result1[i1][i2][0] += 1
        result1[i1][i2].append(edge.tag)
    count_edge1 = None
    return result1, count_edge1


In [4]:
#"C:/Users/gebre/Graph+DSL (4)/Graph+DSL/training"
# "C:/Users/gebre/Graph+DSL (4)/Graph+DSL/training"


In [5]:
## solvers for h and w

def linear(input, target):
    As = [0,1,2,3,4,5, 1/2]
    Bs = [1, -4, -3, -2, -1, 0, 2, 3, 4, 1/2]
    answer = []
    for a in As:
        for b in Bs:
            if a * input + b == target:
                answer = [a,b]
                return answer
    return None

def xnode_h (Xnode):
    if isinstance(Xnode, Pnode) or isinstance(Xnode, Vnode):
        return None
    return get_height(Xnode)

def xnode_w (Xnode):
    if isinstance(Xnode, Pnode) or isinstance(Xnode, Vnode):
        return None
    return get_width(Xnode)

def number_of_colorset(Xnode):
    if isinstance(Xnode, Gnode):
        return len(Xnode.color)
    return None


def node_size(Xnode):
    if isinstance(Xnode, Pnode) or isinstance(Xnode, Vnode):
        return None
    return get_number_of_nodes(Xnode)

def onode_count1(Xnode):
    if isinstance(Xnode, Gnode):
        count = 1
        for onode in Xnode.Onode_list:
            if onode.condition == ("single_color", "get_manhattan_dist") and onode.color != {0}:
                count += 1
        return count
    else :
        return None
    
def onode_count2(Xnode):
    if isinstance(Xnode, Gnode):
        count = 1
        for onode in Xnode.Onode_list:
            if onode.condition == ("multi_color", "get_polar_dist") and onode.color != {0}:
                count += 1
        return count
    else :
        return None
    
def onode_count3(Xnode):
    if isinstance(Xnode, Gnode):
        count = 1
        for onode in Xnode.Onode_list:
            if onode.condition == ("single_color", "get_polar_dist") and onode.color != {0}:
                count += 1
        return count
    else :
        return None
    
def onode_count4(Xnode):
    if isinstance(Xnode, Gnode):
        count = 1
        for onode in Xnode.Onode_list:
            if onode.condition == ("multi_color", "get_manhattan_dist") and onode.color != {0}:
                count += 1
        return count
    else :
        return None

In [6]:
## solver for color set
def number_of_colorsett(Xnode):
    if isinstance(Xnode, Pnode) or isinstance(Xnode, Vnode):
        return None
    return set(Xnode.color)  # Returning the set of colors instead of its length

## you can write function here!

In [7]:
import copy
get_int = [xnode_h, xnode_w, number_of_colorsett, node_size, onode_count1, onode_count2, onode_count3, onode_count4]
def number_of_colorsett(Xnode):
    if isinstance(Xnode, Pnode) or isinstance(Xnode, Vnode):
        return None
    return set(Xnode.color)  # Returning the set of colors instead of its length

def identity_transformation(value, target):  #adding some value t
    if value == target:
        temp = lambda x: number_of_colorsett(x)
        tag = (number_of_colorsett.__name__, 'identity')
        return (temp, tag)
    return None

def multiplicative_transformation(value, target):
    if target % value == 0:
        factor = target // value
        temp = lambda x, f=factor: number_of_colorsett(x) * f
        tag = (number_of_colorsett.__name__, 'multiplicative', factor)
        return (temp, tag)
    return None

def additive_transformation(value, target):
    difference = target - value
    temp = lambda x, d=difference: number_of_colorsett(x) + d
    tag = (number_of_colorsett.__name__, 'additive', difference)
    return (temp, tag)

###used another solver function called color_solver_function for the color set predictor 
#this color_solver_function make use of the functions defined above to determine the transformations(relationships) between the target and the predicted 

def color_solver_function(Xnode, target_set):
    """Create solvers that match the color set of Xnode to the target set."""
    solver = set()
    colors = number_of_colorsett(Xnode)
    if colors is None:
        return solver
    
    # Check if the exact set of colors matches
    if colors.issuperset(target_set):
      temp = lambda x: target_set
      tag = (number_of_colorsett.__name__, 'superset match')
      solver.add((temp, tag))
    if colors.issubset(target_set):
      temp = lambda x: target_set
      tag = (number_of_colorsett.__name__, 'superset match')
      solver.add((temp, tag))
    if colors.issuperset(target_set):
      temp = lambda x: colors
      tag = (number_of_colorsett.__name__, 'superset match')
      solver.add((temp, tag))
    if colors != target_set:
       temp = lambda x: target_set
       tag = (number_of_colorsett.__name__, 'transformed')
       solver.add((temp, tag))
    if colors == target_set:
        temp = lambda x: number_of_colorsett(x)
        tag = (number_of_colorsett.__name__, 'exact match')
        solver.add((temp, tag))
    # Checking if the colors are a subset of the target set
    if colors.issubset(target_set):
        temp = lambda x: number_of_colorsett(x).issubset(target_set)
        tag = (number_of_colorsett.__name__, 'subset match')
        solver.add((temp, tag))

    # Checking if the colors are a superset of the target set
    if colors.issuperset(target_set):
        temp = lambda x: number_of_colorsett(x).issuperset(target_set)
        tag = (number_of_colorsett.__name__, 'superset match')
        solver.add((temp, tag))

    # Checking if there is an intersection between the color sets
    if colors & target_set:
        temp = lambda x: bool(number_of_colorsett(x) & target_set)
        tag = (number_of_colorsett.__name__, 'intersection match')
        solver.add((temp, tag))

    return solver
#No modifications done for this Solver_function
def solver_function(Xnode, target):
    solver = set()
    
    v1 = xnode_h(Xnode)
    if not isinstance(v1, int):
        pass
    else :
        answer1 = linear(v1, target)
        if answer1 != None:
            # temp1 = lambda x: ((xnode_h(x) * answer1[0] + answer1[1] if not xnode_h(x) == None else 0) , print ("lambda : ",temp1  ,"candi(x) :",xnode_h(x),"candi: ", xnode_h.__name__, "answer[0] :", answer1[0], "answer[1] ", answer1[1]))
            temp1 = lambda x: ((xnode_h(x) * answer1[0] + answer1[1] if not xnode_h(x) == None else 0))
            tag1 = (xnode_h.__name__, answer1[0], answer1[1])
            solver.add((temp1,tag1))

    v2 = xnode_w(Xnode)
    if not isinstance(v2, int):
        pass
    else :
        answer2 = linear(v2, target)
        if answer2 != None:
            # temp2 = lambda x: ((xnode_w(x) * answer2[0] + answer2[1] if not xnode_w(x) == None else 0) , print ("lambda : ",temp2  ,"candi(x) :",xnode_w(x),"candi: ", xnode_w.__name__, "answer[0] :", answer2[0], "answer[1] ", answer2[1]))
            temp2 = lambda x: ((xnode_w(x) * answer2[0] + answer2[1] if not xnode_w(x) == None else 0))
            tag2 = (xnode_w.__name__, answer2[0], answer2[1])
            solver.add((temp2,tag2))

    v3 = number_of_colorset(Xnode)
    if not isinstance(v3, int):
        pass
    else :
        answer3 = linear(v3, target)
        if answer3 != None:
            # temp3 = lambda x: ((number_of_colorset(x) * answer3[0] + answer3[1] if not number_of_colorset(x) == None else 0) , print ("lambda : ",temp1  ,"candi(x) :",number_of_colorset(x),"candi: ", number_of_colorset.__name__, "answer[0] :", answer3[0], "answer[1] ", answer3[1]))
            temp3 = lambda x: ((number_of_colorset(x) * answer3[0] + answer3[1] if not number_of_colorset(x) == None else 0))
            tag3 = (number_of_colorset.__name__, answer3[0], answer3[1])
            solver.add((temp3,tag3))

    v4 = node_size(Xnode)
    if not isinstance(v4, int):
        pass
    else :
        answer4 = linear(v4, target)
        if answer4 != None:
            # temp4 = lambda x: ((node_size(x) * answer4[0] + answer4[1] if not node_size(x) == None else 0) , print ("lambda : ",temp4  ,"candi(x) :",node_size(x),"candi: ", node_size.__name__, "answer[0] :", answer4[0], "answer[1] ", answer4[1]))
            temp4 = lambda x: ((node_size(x) * answer4[0] + answer4[1] if not node_size(x) == None else 0))
            tag4 = (node_size.__name__, answer4[0], answer4[1])
            solver.add((temp4,tag4))

    v5 = onode_count1(Xnode)
    if not isinstance(v5, int):
        pass
    else :
        answer5 = linear(v5, target)
        if answer5 != None:
            # temp5 = lambda x: ((onode_count1(x) * answer5[0] + answer5[1] if not onode_count1(x) == None else 0) , print ("lambda : ",temp5  ,"candi(x) :",onode_count1(x),"candi: ", onode_count1.__name__, "answer[0] :", answer5[0], "answer[1] ", answer5[1]))
            temp5 = lambda x: ((onode_count1(x) * answer5[0] + answer5[1] if not onode_count1(x) == None else 0))
            tag5 = (onode_count1.__name__, answer5[0], answer5[1])
            solver.add((temp5,tag5))

    v6 = onode_count2(Xnode)
    if not isinstance(v6, int):
        pass
    else :
        answer6 = linear(v6, target)
        if answer6 != None:
            # temp6 = lambda x: ((onode_count2(x) * answer6[0] + answer6[1] if not onode_count2(x) == None else 0) , print ("lambda : ",temp6  ,"candi(x) :",onode_count2(x),"candi: ", onode_count2.__name__, "answer[0] :", answer6[0], "answer[1] ", answer6[1]))
            temp6 = lambda x: ((onode_count2(x) * answer6[0] + answer6[1] if not onode_count2(x) == None else 0))
            tag6 = (onode_count2.__name__, answer6[0], answer6[1])
            solver.add((temp6,tag6))

    v7 = onode_count3(Xnode)
    if not isinstance(v7, int):
        pass
    else :
        answer7 = linear(v7, target)
        if answer7 != None:
            # temp7 = lambda x: ((onode_count3(x) * answer7[0] + answer7[1] if not onode_count3(x) == None else 0) , print ("lambda : ",temp7  ,"candi(x) :",onode_count3(x),"candi: ", onode_count3.__name__, "answer[0] :", answer7[0], "answer[1] ", answer7[1]))
            temp7 = lambda x: ((onode_count3(x) * answer7[0] + answer7[1] if not onode_count3(x) == None else 0))
            tag7 = (onode_count3.__name__, answer7[0], answer7[1])
            solver.add((temp7,tag7))

    v8 = onode_count4(Xnode)
    if not isinstance(v8, int):
        pass
    else :
        answer8 = linear(v8, target)
        if answer8 != None:
            # temp8 = lambda x: ((onode_count4(x) * answer8[0] + answer8[1] if not onode_count4(x) == None else 0) , print ("lambda : ",temp8  ,"candi(x) :",onode_count4(x),"candi: ", onode_count4.__name__, "answer[0] :", answer8[0], "answer[1] ", answer8[1]))
            temp8 = lambda x: ((onode_count4(x) * answer8[0] + answer8[1] if not onode_count4(x) == None else 0))
            tag8 = (onode_count4.__name__, answer8[0], answer8[1])
            solver.add((temp8,tag8))

    return solver

    

In [8]:
#I didnot addd modifications to this extractor function 
def extractor(KG, adj, target_xnode):
    node_info = {}
    node_info["type"] = target_xnode.type
    node_info["properties"] = set()

    if isinstance(target_xnode, Gnode):
        node_info["definition"] = "Gnode"
        return node_info
    else :
        node_info["definition"] = target_xnode.condition

    # edge_list = KG[1]
    edge_dict = {}
    # print("node type : ", type(target_xnode))

    # print("preparing edge list... ")
    # result, _ = print_adj(KG)
    index = KG[0].index(target_xnode)
    for i, n in enumerate(KG[0]):
        if n.output == 1:
            break
    index2 = i
    for edge in adj[index:index2]:
        if edge[0] == 0:
            continue
        # print(edge[1])
        for tag in edge[1:]:
            # print("tag : ", tag)
            edge_dict[tag[0]] = edge[0]
    # print("\rfinish edge list")

    max_size = 0
    min_size = 900
    unique = 1
    for node in KG[0]:
        if node.type == "Onode" and node.condition == target_xnode.condition and node.input == 1:
            if node.color == target_xnode.color and node != target_xnode:
                unique = 0
            max_size = max(max_size, get_number_of_nodes(node))
            min_size = min(max_size, get_number_of_nodes(node))
    if max_size <= get_number_of_nodes(target_xnode):
        node_info["properties"].add("max_size")
    if min_size >= get_number_of_nodes(target_xnode):
        node_info["properties"].add("min_size")
    if unique == 1:
        node_info["properties"].add("unique_color")

    if "is_ring" in edge_dict.keys():
        node_info["properties"].add("ring_shape")
    if "is_rectangle" in edge_dict.keys():
        node_info["properties"].add("rectangle_shape")
    if "is_square" in edge_dict.keys():
        node_info["properties"].add("square_shape")
    if "is_symmetric" in edge_dict.keys():
        node_info["properties"].add("symmetric")

    node_info["edge_list"] = edge_dict
    return node_info


In [9]:
from itertools import combinations
#there is no any modifications done to this specifier function 
def specifer(KG, adj,target_xnode):  ## 다시 짜야할 것 같다. -> 람다함수 말고 그냥 유일하게 가지고 있는 특성들을 리스트로 반환하는 함수
    # adj, _ = print_adj(KG)
    node_info = extractor(KG, adj, target_xnode)
    speic = []
    if node_info["type"] == "Gnode":
        speic.append(("definition", "Gnode"))
        return tuple(speic)
    
    candi_node_list = []
    for n in KG[0]:
        if n.input  == 1 and (n.type == "Onode" or n.type == "Gnode") :
            candi_node_list.append(extractor(KG, adj, n))
    
    obj_def = node_info["definition"]
    speic.append(("definition", obj_def))
    
    for property in node_info["properties"]:
        speic.append(("properties", property))


    all_combinations = []
    for r in range(1, len(speic) + 1):
        all_combinations.extend(combinations(speic, r))

    # print("properties of target_node :", speic)
    for i, combo in enumerate(all_combinations):
        # print("combo :", combo)
        t_node = []
        for candi_info in candi_node_list:
            # print("candi_info :", candi_info)
            match = 1
            for c in combo :
                # print(c[0], c[1])
                if c[0] == "definition":
                    if c[1] != candi_info[c[0]] :   
                        match = 0    
                else :
                    if c[1] not in candi_info[c[0]]:
                        match = 0  
            if match == 1:
                t_node.append(n)
                # print("추가됨")
        # print("len of t_node :", len(t_node))
        if len(t_node) == 1:
            return combo
        else :
            continue
    return None

In [10]:
def speci_tester(s, n_info):
    for tag in s[0]:
        if tag[0] == "definition":
            if tag[1] != n_info["definition"] :   
                return False
        else :
            if tag[1] not in n_info["properties"] : 
                return False
    return True
def Solution_generaterrs(task):
    KG = Make_KG(task)
    pair_1 = KG[0]
    for n in KG[0][0]:
        if isinstance(n, Gnode) and n.output == 1:
            target = (get_height(n), get_width(n), number_of_colorsett(n))   ###added the number_of_colorsett to get the target colors
    
    node_solver_h = set()
    node_solver_w = set()
    node_solver_color = set()
    
    for n in pair_1[0]:   
        if isinstance(n, Onode) or isinstance(n, Gnode):
            if n.output == 1:
                continue
            # ans_h = solver_function(n, target[0])
            # if ans_h:
            #     for a in ans_h:
            #         node_solver_h.add((n, a))
                    
            # ans_w = solver_function(n, target[1])
            # if ans_w:
            #     for a in ans_w:
            #         node_solver_w.add((n, a))
            
            ans_color = color_solver_function(n, target[2])
            if ans_color:
                for a in ans_color:
                    node_solver_color.add((n, a))
    
    solver_h = set()
    solver_w = set()
    solver_color = set()
    # for s in node_solver_h:
    #     print("type : ",s[0].type , "func : ", s[1][1])
    #     if isinstance(s[0], Gnode):
    #         print(s[1][0](s[0]))
    # print("#############################################")
    adj,_ = print_adj(KG[0])
    print("solver without spicifer : ",len(node_solver_h) , len(node_solver_w), len(node_solver_color))

    # print("generating solver for h")
    # for i, s_h in enumerate(node_solver_h):
    #     progress = (i + 1) / len(node_solver_h) * 100
    #     bar_width = int(progress)
    #     print('\r [%-*s] %.1f%%' % (100, '=' * bar_width, progress), end='')
    #     temp = specifer(KG[0], adj, s_h[0])    
    #     if temp:
    #         temp = tuple(temp)
    #     else:
    #         continue
    #     solver_h.add((temp, s_h[1]))
    # print("\ngenerating solver for w")
    # for i, s_w in enumerate(node_solver_w):
    #     progress = (i + 1) / len(node_solver_w) * 100
    #     bar_width = int(progress)
    #     print('\r [%-*s] %.1f%%' % (100, '=' * bar_width, progress), end='')
    #     temp = specifer(KG[0],adj, s_w[0])    
    #     if temp:
    #         temp = tuple(temp)
    #     else:
    #         continue
    #     solver_w.add((temp, s_w[1]))
    print("\ngenerating solver for c")
    for i, s_color in enumerate(node_solver_color):
        progress = (i + 1) / len(node_solver_color) * 100
        bar_width = int(progress)
        print('\r [%-*s] %.1f%%' % (100, '=' * bar_width, progress), end='')
        temp = specifer(KG[0],adj, s_color[0])
        if temp:
            temp = tuple(temp)
        else:
            continue
        solver_color.add((temp, s_color[1]))
    
    print("\nsolver num:", len(solver_h), len(solver_w), len(solver_color)) 
    for i in range(1, len(KG)):
        adj, _ = print_adj(KG[i])
        candi_node = []
        candi_node_info = []
        for n in KG[i][0]:
            if isinstance(n, Gnode) and n.output == 1:
                target_i = (get_height(n), get_width(n), number_of_colorsett(n))    ####addedd the function for getting the number of color set 
            if (n.type == "Gnode" or n.type == "Onode") and n.input == 1:
                candi_node_info.append(extractor(KG[i], adj, n))
                candi_node.append(n)
                
        new_solver_h = set()
        new_solver_w = set()
        new_solver_color = set()
        
        # for k, s in enumerate(solver_h):
        #     for j, n_info in enumerate(candi_node_info):
        #         if speci_tester(s, n_info) == False:
        #             continue
        #         if s[1][0](candi_node[j]) == target_i[0]:
        #             new_solver_h.add(s)
        #             break
        # solver_h = new_solver_h
        # for k, s in enumerate(solver_w):
        #     for j, n_info in enumerate(candi_node_info):
        #         if speci_tester(s, n_info) == False:
        #             continue
        #         if s[1][0](candi_node[j]) == target_i[1]:
        #             new_solver_w.add(s)
        #             break
        # solver_w = new_solver_w
        
        for k, s in enumerate(solver_color):
            for j, n_info in enumerate(candi_node_info):
                if speci_tester(s, n_info) == False:
                    continue
                if s[1][0](candi_node[j]) == target_i[2]:
                    new_solver_color.add(s)
                    break 
        solver_color = new_solver_color   #### for Color 

        print("after ",i+1,"th pair solver num:", len(solver_h), len(solver_w), len(solver_color))
    print("finish finding possible solvers")
    return solver_h, solver_w, solver_color

In [11]:
#this function now also contains for the color set count 
def GridSize_pridictorr(task):
    solver_h, solver_w, solver_color = Solution_generaterrs(task)
    KG_test = Make_test_KG(task)
    adj, _ = print_adj(KG_test)
    candi_node = []
    candi_node_info = []
    candi_answer_h = []
    candi_answer_w = []
    candi_answer_color = []  #####
    
    for n in KG_test[0]:
        if n.type == "Onode" or n.type == "Gnode":
            candi_node.append(n)
            candi_node_info.append(extractor(KG_test, adj, n))
            
    for s in solver_h:
        for i, n in enumerate(candi_node):
            if speci_tester(s, candi_node_info[i]):
                candi_answer_h.append(s[1][0](n))
                
    for s in solver_w:
        for i, n in enumerate(candi_node):
            if speci_tester(s, candi_node_info[i]):
                candi_answer_w.append(s[1][0](n))
                
    for s in solver_color:
        for i, n in enumerate(candi_node):
            if speci_tester(s, candi_node_info[i]):
                candi_answer_color.append(s[1][0](n))
                
    return candi_answer_h, candi_answer_w, candi_answer_color  ####

In [13]:
import os
import json

correct_list_c = []
wrong_list_c = []

files = os.listdir("./data/training/")
index = 0
# Loop through each file
for index, f in enumerate(files):
    file_path = "./data/training/" + f  
    try:
        with open(file_path) as file:
            task = json.load(file)
            
        print(f"Processing file: {f} ({index + 1}/{len(files)})")

        # Extract truth color set
        truth_color_set = {color for row in task["test"][0]["output"] for color in row}

        # Call function to predict color sets
        candi_answer_h, candi_answer_w, candi_answer_color_sets = GridSize_pridictorr(task)
        
        print(f"Found {len(candi_answer_color_sets)} color set answers")
        print(f"Truth Color Set: {truth_color_set}")
        print("Predicted Color Sets:")
        for color_set in candi_answer_color_sets:
            print(color_set)
        
        # Check if any predicted color set matches truth
        color_correct = any(color_set == truth_color_set for color_set in candi_answer_color_sets)
        if color_correct:
            print("Color Set Correct!")
            correct_list_c.append(f)
        else:
            print("Color Set Incorrect!")
            wrong_list_c.append(f)
        
    except Exception as e:
        print(f"Error processing file: {f}")
        print(e)

# Print results
print(f"Correct: {len(correct_list_c)}, Wrong: {len(wrong_list_c)}")


Processing file: a85d4709.json (1/400)
converting task to knowledge graph... 1 / 4
 [====================================================================================================] 100.0%
converting task to knowledge graph... 2 / 4
 [====================================================================================================] 100.0%
converting task to knowledge graph... 3 / 4
 [====================================================================================================] 100.0%
converting task to knowledge graph... 4 / 4
 [====================================================================================================] 100.0%
 
solver without spicifer :  0 0 15

generating solver for c
 [====================================================================================================] 100.0%
solver num: 0 0 7
after  2 th pair solver num: 0 0 0
after  3 th pair solver num: 0 0 0
after  4 th pair solver num: 0 0 0
finish finding possible solvers
Found 0 color 

KeyboardInterrupt: 